# Initiate Jupyter Notebook

another test commit

In [4]:
import pandas as pd

In [7]:
import mysql.connector as sql

mydb = sql.connect(
  host="localhost",
  user="root",
  passwd="NYg1@nts",
  database="NRL_data"
)

player_df = pd.read_sql_query("SELECT * FROM Players;",
    con=mydb)
player_df.head()

,id,first_name,last_name,is_active,current_team,height_cm,weight_kg,date_of_birth,city,state,country,url,previous_team1,previous_team2,previous_team3
0,1,Morgan,Harper,1.0,3,NaN,NaN,None,None,None,None,None,NaN,None,None
1,2,Alex,Glenn,1.0,1,182.0,101.0,None,Auckland,None,NZ,None,NaN,None,None
2,3,Andrew,McCullough,1.0,1,180.0,90.0,None,Dalby,QLD,AUS,None,NaN,None,None
3,4,David,Fifita,1.0,1,186.0,107.0,None,Brisbane,QLD,AUS,None,NaN,None,None
4,5,George,Fai,1.0,1,187.0,111.0,None,Brisbane,QLD,AUS,None,NaN,None,None
